In [63]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn import metrics

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import torch
from torch import nn
import torch.nn.functional as F

In [64]:
def classification_metrics(target, pred):
    tn, fp, fn, tp = metrics.confusion_matrix(target, pred).ravel()
    acc = (tp + tn) / (tn + fp + fn + tp)
    sen = tp / (tp + fn)
    spc = tn / (tn + fp)
    prc = tp / (tp + fp)
    return acc, sen, spc, prc

In [65]:
data = load_breast_cancer()
X, y = data['data'], data['target']
X.shape, y.shape

((569, 30), (569,))

In [66]:
X_norm = (X - X.mean(axis=0)) / X.std(axis=0)
X_train, X_test, y_train, y_test = train_test_split(X_norm, y)

In [67]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(dim=1)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(dim=1)

In [68]:
X_train.shape, X_test.shape

(torch.Size([426, 30]), torch.Size([143, 30]))

In [69]:
class MyMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        y = self.linear1(x)
        y = self.relu(y)
        y = self.linear2(y)
        y = self.sigmoid(y)
        return y

In [70]:
input_dim, hidden_dim, output_dim = 30, 10, 1
model = MyMLP(input_dim, hidden_dim, output_dim)

In [71]:
loss_function = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [72]:
max_epock = 200

for i in range(max_epock):
    model.train()
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = loss_function(y_pred, y_train)
    loss.backward()
    optimizer.step()
    

In [73]:
model

MyMLP(
  (linear1): Linear(in_features=30, out_features=10, bias=True)
  (linear2): Linear(in_features=10, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)

In [76]:
model.eval()
y_pred = model(X_test)
y_pred = (y_pred > 0.5).float()
y_pred[0:10]

tensor([[0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.]])

In [77]:
classification_metrics(y_test, y_pred)

(0.993006993006993, 0.9888888888888889, 1.0, 1.0)